### **O bloco abaixo seta os endereços dos containeres das camadas de armazenamento**

In [0]:
#Cria um dicionário com os paths dos storage de cada camada a partir de um path padrão
tiers = ['bronze-layer','silver-layer','gold-layer']
adls_paths = {tier: f"abfss://{tier}@medallionapistorage.dfs.core.windows.net/" for tier in tiers}
print(adls_paths)

#Cria as variáveis contendo os paths de cada camada
bronze_adls = adls_paths['bronze-layer']
silver_adls = adls_paths['silver-layer']
gold_adls = adls_paths['gold-layer']

#Acessa os paths para verificar se estão corretos
dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)    

In [0]:
#Cria as variáveis contendo os paths de cada camada
bronze_adls = adls_paths['bronze-layer']
silver_adls = adls_paths['silver-layer']
gold_adls = adls_paths['gold-layer']

In [0]:
#Acessa os paths para verificar se estão corretos
dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)   

### Camada de extração dos dados da API
Os dados são extraídos, considerando a paginação e o máximo de registros retornados por página e persistidos em seu formato original (.json) no storage **bronze**.

In [0]:
import requests
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType


# Cria uma SparkSession
spark = SparkSession.builder.appName("Open Brewery").getOrCreate()

# Seta as variáveis iniciais para a requisição da API 
page=1 
per_page=200
full_response=[]

# Faz a requisição GET para a API
while True:
    url = f"https://api.openbrewerydb.org/v1/breweries?page={page}&{per_page}"
    print(f"Buscando a página {page} da API")
    try:
        response = requests.get(url)

        #Verifica se a resposta foi bem sucedida
        response.raise_for_status()
        data = response.json()

        if not data:
            print("Não foram retornados dados na resposta")
            break
        else:
            #Salva todas as respostas em uma lista 
            full_response.extend(data)
    except Exception as e:
        print(f"Ocorreu um erro durante a requisição: {e}")
        break
    
    page+=1

#Indica o arquivo e salva os dados no formato json (de origem) no storage indicado
file_path = f"{bronze_adls}open_breweries_raw.json" 

df = spark.createDataFrame(full_response)
df.write.mode("overwrite").json(file_path)


print(f"Os dados foram salvos no storage {file_path}")
